#Libraries and Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#change directory
%cd /content/drive/MyDrive/Time Series Forecasting Product

/content/drive/MyDrive/Time Series Forecasting Product


In [ ]:
#installing libraries
!pip install pyyaml==5.4.1
!pip install darts

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 636 kB 7.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 400 kB 9.8 MB/s 
     |████████████████████████████████| 43 kB 2.5 MB/s 
     |████████████████████████████████| 179 kB 61.6 MB/s 
     |████████████████████████████████| 9.8 MB 55.1 MB/s 
     |████████████████████████████████| 585 kB 65.3 MB/s 
     |████████████████████████████████| 11.2 MB 14.8 MB/s 
     |████████████████████████████████| 1.4 MB 60.4 MB/s 
     |████████████████████████████████| 793 kB 67.8 MB/s 
     |████████████████████████████████| 381 kB 69.0 MB/s 
     |████████████████████████████████| 944 kB 50.0 MB/s 
     |████████████

In [ ]:
#libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from darts.timeseries import TimeSeries
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [ ]:
#load the data
#YYYY-MM-DD
df = pd.read_csv('nyc_data.csv', index_col = 0, parse_dates = True)
future_df = pd.read_csv('future.csv', index_col = 0, parse_dates = True)

In [ ]:
df.head(0)

,Demand,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,


In [ ]:
#Extract regressors
X_train = df.iloc[:,1:]
X_future = future_df.iloc[:,1:]

In [ ]:
#merge both
X = pd.concat([X_train, X_future])

In [ ]:
#Rename variable
df = df.rename(columns = {'Demand': 'y'})
df.head(0)

,y,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,


#Prepare for LSTM

In [ ]:
#Time Series object
series = TimeSeries.from_series(df.y)
covariates = TimeSeries.from_dataframe(X)

In [ ]:
#year
year_series = datetime_attribute_timeseries(
    pd.date_range(start=series.start_time(),
                  freq = series.freq_str,
                  periods= X.shape[0]),
                  attribute ="year",
                  one_hot = False)

#month
month_series = datetime_attribute_timeseries(year_series,
                                            attribute ="month",
                                            one_hot = True)

#weekday
weekday_series = datetime_attribute_timeseries(year_series,
                                               attribute ="weekday",
                                               one_hot = True)

In [ ]:
# preparing scalers
transformer1 = Scaler()
transformer2 = Scaler()

In [ ]:
#Scale the Y
y_transformed = transformer1.fit_transform(series)

In [ ]:
# Scale the covariates
covariates = covariates.stack(year_series)
covariates_transformed = transformer2.fit_transform(covariates)
covariates_transformed = covariates_transformed.stack(month_series)
covariates_transformed = covariates_transformed.stack(weekday_series)

#LSTM

In [ ]:
#get the best parameters
parameters  = pd.read_csv("Forecasting Product/best_params_lstm.csv", 
                          index_col = 0)
parameters

,14
dropout,0.200000
hidden_dim,30.000000
input_chunk_length,20.000000
lr,0.003000
n_epochs,20.000000
n_rnn_layers,4.000000
training_length,20.000000
rmse,83.370327


In [ ]:
#getting the parameters
n_rnn_layers = int(parameters.loc["n_rnn_layers"][0])
dropout = float(parameters.loc["dropout"][0])
hidden_dim = int(parameters.loc["hidden_dim"][0])
input_chunk_length = int(parameters.loc["input_chunk_length"][0])
lr = float(parameters.loc["lr"][0])
n_epochs = int(parameters.loc["n_epochs"][0])
training_length = int(parameters.loc["training_length"][0])

In [ ]:
#model
#https://unit8co.github.io/darts/generated_api/darts.models.forecasting.rnn_model.html
model = RNNModel(model = "LSTM",
                 hidden_dim = hidden_dim,
                 n_rnn_layers= n_rnn_layers,
                 dropout = dropout,
                 n_epochs = n_epochs,
                 optimizer_kwargs = {"lr": lr},
                 random_state = 1502,
                 training_length = training_length,
                 input_chunk_length= input_chunk_length,
                 pl_trainer_kwargs = {"accelerator": "gpu", "gpus": -1,
                                      "auto_select_gpus": True},
                 save_checkpoints = True)

In [ ]:
#fit the model to the data
model.fit(y_transformed,
          future_covariates = covariates_transformed)

2022-07-08 14:40:57 darts.models.forecasting.torch_forecasting_model INFO: Train dataset contains 2172 samples.
2022-07-08 14:40:57 darts.models.forecasting.torch_forecasting_model INFO: Time series values are 64-bits; casting model to float64.
2022-07-08 14:41:09 pytorch_lightning.trainer.connectors.accelerator_connector INFO: Auto select gpus: [0]
2022-07-08 14:41:09 pytorch_lightning.utilities.rank_zero INFO: GPU available: True, used: True
2022-07-08 14:41:09 pytorch_lightning.utilities.rank_zero INFO: TPU available: False, using: 0 TPU cores
2022-07-08 14:41:09 pytorch_lightning.utilities.rank_zero INFO: IPU available: False, using: 0 IPUs
2022-07-08 14:41:09 pytorch_lightning.utilities.rank_zero INFO: HPU available: False, using: 0 HPUs
2022-07-08 14:41:09 pytorch_lightning.accelerators.gpu INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
2022-07-08 14:41:09 pytorch_lightning.callbacks.model_summary INFO: 
  | Name          | Type             | Params
-----------------------------

Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:378: UserWarning: `ModelCheckpoint(monitor='val_loss')` could not find the monitored key in the returned metrics: ['train_loss', 'epoch', 'step']. HINT: Did you call `log('val_loss', value)` in the `LightningModule`?
  warning_cache.warn(m)


#Predictions and exporting

In [ ]:
predictions_lstm = model.predict(n = len(future_df),
                                 future_covariates = covariates_transformed)
predictions_lstm = TimeSeries.pd_series(
    transformer1.inverse_transform(predictions_lstm)).rename("lstm")
predictions_lstm

In [ ]:
#exporting
predictions_lstm.to_csv("Forecasting Product/Ensemble/predictions_lstm.csv")